In [1]:
import os
import dlib
import cv2
import numpy as np

##   Initialising Dlib Predictor

In [2]:
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

## Model Points for a 3D Face

In [6]:
model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip
    (0.0, -330.0, -65.0),        # Chin
    (-225.0, 170.0, -135.0),     # Left eye left corner
    (225.0, 170.0, -135.0),    # Right eye right corner
    (-150.0, -150.0, -125.0),    # Left Mouth corner
    (150.0, -150.0, -125.0)     # Right mouth corner
], dtype=np.float64)     

## Auto Rotate the Image if its Rotated

In [7]:
def rotate_image(image, angle):
    if angle == 0:
        return image
    elif angle == 90:
        return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        return cv2.rotate(image, cv2.ROTATE_180)
    elif angle == 270:
        return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        raise ValueError("Angle must be 0, 90, 180, or 270")


## Get the 2D image coordinates from facial landmarks

In [8]:
def get_image_points(shape):
    
    return np.array([
        (shape.part(30).x, shape.part(30).y),     # Nose tip
        (shape.part(8).x, shape.part(8).y),       # Chin
        (shape.part(36).x, shape.part(36).y),     # Left eye left corner
        (shape.part(45).x, shape.part(45).y),     # Right eye right corner
        (shape.part(48).x, shape.part(48).y),     # Left Mouth corner
        (shape.part(54).x, shape.part(54).y)      # Right mouth corner
    ], dtype="double")

def draw_axes(img, rotation_vector, translation_vector, camera_matrix, dist_coeffs, nose_point):
    axis_length = 100
    axes_3D = np.float32([
        [axis_length, 0, 0],  # X - Red
        [0, axis_length, 0],  # Y - Green
        [0, 0, axis_length]   # Z - Blue
    ]).reshape(-1, 3)

    imgpts, _ = cv2.projectPoints(axes_3D, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    imgpts = np.int32(imgpts).reshape(-1, 2)

    origin = tuple(int(x) for x in nose_point)
    cv2.line(img, origin, tuple(imgpts[0]), (0, 0, 255), 3)  # X-axis
    cv2.line(img, origin, tuple(imgpts[1]), (0, 255, 0), 3)  # Y-axis
    cv2.line(img, origin, tuple(imgpts[2]), (255, 0, 0), 3)  # Z-axis


In [9]:

def get_yaw_pitch_roll(image_path):
      
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError("Image not found.")

    try:

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Dlib needs gray images

        # Detect faces
        faces = detector(gray)
        if len(faces) == 0:
            print("No face detected.")
            return "","No Landmarks",None,None,None

        if len(faces)>1:
            print("More than one face detected.")
            
            
        # Use the first detected face
        faces = sorted(faces, key=lambda r: r.width() * r.height(), reverse=True)

        face = faces[0]
        landmarks = predictor(gray, face)

        # 2D image points
        image_points = get_image_points(landmarks)
        # image_points = np.array([
        #     (landmarks.part(30).x, landmarks.part(30).y),  # Nose tip
        #     (landmarks.part(8).x, landmarks.part(8).y),    # Chin
        #     (landmarks.part(36).x, landmarks.part(36).y),  # Left eye left corner
        #     (landmarks.part(45).x, landmarks.part(45).y),  # Right eye right corner
        #     (landmarks.part(48).x, landmarks.part(48).y),  # Left mouth corner
        #     (landmarks.part(54).x, landmarks.part(54).y)   # Right mouth corner
        # ], dtype="double")


        # Camera internals
        height, width = image.shape[:2]
        focal_length = width
        center = (width / 2, height / 2)
        camera_matrix = np.array([
            [focal_length, 0, center[0]],
            [0, focal_length, center[1]],
            [0, 0, 1]
        ], dtype="double")

        dist_coeffs = np.zeros((4, 1))  # Assuming no distortion

        # Solve PnP
        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        if not success:
            print("PnP solution failed.")
            return None, None, None


        # Convert rotation vector to matrix
        rotation_matrix, _ = cv2.Rodrigues(rotation_vector)

        # Compose pose matrix
        pose_matrix = np.hstack((rotation_matrix, translation_vector))

        # Decompose to Euler angles
        _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_matrix)

        pitch, yaw, roll = [abs(float(angle)) for angle in euler_angles]

        # pitch, yaw, roll = euler_angles.flatten()
        # pitch, yaw, roll = abs(pitch), abs(yaw), abs(roll)

        # # pitch, yaw, roll = [abs(float(angle)) for angle in euler_angles]
        # pitch = abs(np.degrees(pitch))
        # yaw = abs(np.degrees(yaw))
        # roll = abs(np.degrees(roll))

        if pitch>100:
            pitch = 180 - pitch
        if yaw>100:
            yaw = 180 - yaw
        if roll>100:
            roll = 180 - roll

        # Display results
        print(f"Yaw: {yaw:.2f}°")
        print(f"Pitch: {pitch:.2f}°")
        print(f"Roll: {roll:.2f}°")

        
        return pitch,yaw,roll
        
    except Exception as e:
        print(f"Error : {e}")
        return None,None,None

In [10]:
image_path = "BruceLee.jpg"
get_yaw_pitch_roll(image_path)

Yaw: 2.91°
Pitch: 0.67°
Roll: 2.20°


/var/folders/9y/wvqwglpd2kn80kp48cpw_g7w0000gn/T/ipykernel_45166/1288215083.py:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pitch, yaw, roll = [abs(float(angle)) for angle in euler_angles]


(0.673810942301543, 2.9054339844164088, 2.2036180075887573)